In [1]:
import os

import numpy as np
import pandas as pd

print(os.getcwd())
root_path = '/home/lyq/DataSet/FakeNews/gossipcop'
os.chdir(root_path)
print(os.getcwd())


/home/lyq/PycharmProjects/QwenVLRationaleGenerate/data/gossipcop
/home/lyq/DataSet/FakeNews/gossipcop


In [2]:
rationale_names = ['td','cs']
df = pd.read_csv('gossipcop.csv')
df.rename(columns={'id':'source_id'}, inplace=True)
rationale_dfs = [ pd.read_csv(f'{rationale_name}.csv')  for rationale_name in rationale_names]



In [3]:
for i, r_df in enumerate(rationale_dfs):
    rationale_dfs[i] = r_df.merge(df[['source_id', 'label']], how='left', on='source_id')
    rationale_dfs[i]['acc'] = rationale_dfs[i].apply(lambda row: int(row['label'] == row['authenticity']), axis=1)
    print((rationale_dfs[i]['acc'] == 1).sum() / len(rationale_dfs[i]))


0.8341712617580279
0.8359419349606683


In [4]:
from sklearn.model_selection import train_test_split


def split_rationale_df(df, test_ratio=0.2, random_state=None, stratify_col=None):

    # 分层抽样的依据列（如果指定了）
    stratify = df[stratify_col] if stratify_col else None
    train_df,test_df = train_test_split(df,test_size=test_ratio,random_state=random_state,stratify=stratify)
    train_id_set,test_id_set = set(train_df['source_id'].tolist()),set(test_df['source_id'].tolist())
    df['split'] = df['source_id'].apply(lambda x: 'train' if x in train_id_set else 'test')
    return df


for r_df in rationale_dfs:
    print(r_df.columns)
    r_df = split_rationale_df(r_df,random_state=666,stratify_col='acc')
    print(f'sum: {r_df.shape[0]} train : {(r_df["split"]=="train").sum()} test : {(r_df["split"]=="test").sum()} ')





Index(['source_id', 'authenticity', 'reason', 'label', 'acc'], dtype='object')
sum: 12332 train : 9865 test : 2467 
Index(['source_id', 'authenticity', 'reason', 'label', 'acc'], dtype='object')
sum: 12331 train : 9864 test : 2467 


In [5]:
for i in range(len(rationale_names)):
    rationale_dfs[i].to_csv(f'{rationale_names[i]}.csv',index=False)